In [1]:
## IMPORTS
import numpy as np
import timeit
start = timeit.default_timer()
from datetime import datetime, date
from stage import getStage
import pandas as pd
from datetime import timedelta

In [2]:
nasdaqList = pd.read_pickle("stockData/tickerList.pkl")
test = []
train = []

In [3]:
def calculateGroupReturn(list):

    monthlyReturn = pd.read_pickle("transactionTemplate.pkl")
    monthlyReturn.to_csv("transactionTemplate.csv")
    monthlyReturn = monthlyReturn.resample('M').mean()
    monthlyReturn.index = monthlyReturn.index.strftime("%Y:%m")
    currentColumn = -1
    for symbol in list:
        
        if symbol in monthlyReturn.columns:
            continue
        try:
            df = pd.read_pickle("stockData/nyseNasdaq/"+symbol+".pkl")
            if df == []:
                continue
        except:
            continue
        if not (df['close'] != 0).all():
            continue
        monthlyReturn[symbol] = 1
        currentColumn += 1
        monthCheck = df.index[0].strftime("%Y:%m")
        lastPrice = df.iat[0,0]
        initialPrice = df.iat[0,0]
        for index, element in df.iterrows():
            if (index.strftime("%Y:%m") != monthCheck):
                if (lastPrice / initialPrice >= 1.01 and lastPrice / initialPrice < 2):
                    rowNumber = monthlyReturn.index.get_loc(index.strftime("%Y:%m"))
                    monthlyReturn.iat[rowNumber, currentColumn] = (lastPrice / initialPrice)
                initialPrice = element[0]
                lastPrice = element[0]
                monthCheck = index.strftime("%Y:%m")
            lastPrice = element[0]
    return monthlyReturn
            
import warnings
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning) 
i = 0
listOfDf = calculateGroupReturn(nasdaqList)

In [4]:
indexNum = 0
sum = [1,1]
for i in range(2):
    if i == 0:
        l = train
    else:
        l = test
    index = 0
    while index != len(l):
        if l[index] not in listOfDf.columns:
            l = np.delete(l, index)
        else:
            index += 1
    for index, element in listOfDf[l].iterrows():
        #print(element.to_list())
        listOfStockRet = element.to_list()
        while 1.0 in listOfStockRet:
            listOfStockRet.remove(1.0)
        if len(listOfStockRet) != 0:
            sum[i] = sum[i] * np.mean(listOfStockRet)     
for i in range(2):
    sum[i] = sum[i] / sum[1]
print(sum)

[1.0, 1.0]


In [5]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
## CONSTANTS
HOLDING = 0
TOTAL = 1
def evalOneMax(params):
    print(params)
    now = datetime.now()
    now = now.strftime("%Y-%m-%d-%H:%M")
    sectorDfList = []
    for sector in SectorDict.values():
        sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
        goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
        goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
        goodSectorDf.index = sectorDfList[0].index
        index = 0
        for date in sectorDfList[0].index:
            listOfSector = []
            for df in sectorDfList:
                try:
                    # print(df.at[date,'MA30Slope'],df.at[date,'RS'])
                    # print(individual[0][9],individual[0][10])
                    if  df.at[date,'MA30Slope'] > params[9] or df.at[date,'RS'] > params[10]:
                        if df.at[date,'close'] > df.at[date,'MA30']:
                            # print(df.at[date,'ticker'])
                            listOfSector.append(df.at[date,'ticker'])
                except: 
                    pass
                # print(listOfSector)
            goodSectorDf.iat[index,0] = set(listOfSector)
            index = index + 1
    transactionFit = pd.read_pickle("transactionTemplate.pkl")
    transactionFit['holding'] = np.empty((len(transactionFit), 0)).tolist()
    #print("Reach #1")
    tickers = pd.read_pickle('stockData/tickerList.pkl')
    for symbol in nasdaqList:
        df = getStage(symbol,params,goodSectorDf)
        buyDf = pd.read_pickle('stockData/nyseNasdaq/'+symbol+'Buy.pkl')
        inStage = False
        buyTwice = False
        if df.empty:
            continue
        for index, element in df.iterrows():
            open = 0
            monday = index + timedelta(3)
            i = transactionFit.index.get_loc(index)
            if element.Stage == "Stage 2" or element.Stage == "Buy":
                if monday in buyDf.index:
                    open = buyDf.at[monday,'open']
                else:
                    open = element.close
                #delete
                transactionFit.iat[int(i),HOLDING].append((symbol,open,0))
                if buyTwice:
                    transactionFit.iat[i, HOLDING].append((symbol,open,0))
                if element.Stage == "Buy" and inStage:
                    transactionFit.iat[i, HOLDING].append((symbol,open,0))
                    buyTwice = True
                inStage = True
                continue
            if "Sell" == element.Stage[0:4]:
                if monday in buyDf.index:
                    open = buyDf.at[monday,'open']
                else:
                    open = element.close
                transactionFit.iat[i, HOLDING].append((symbol,open,-1))
                if buyTwice:
                    transactionFit.iat[i, HOLDING].append((symbol,open,-1))
                inStage = False
                buyTwice = False
    #print("Reach #2")
    total = 100
    first = True
    transactionFitCopy = transactionFit
    transactionFitCopy['total'] = 0
    #print("Reach #3")
    for index, element in transactionFitCopy.iterrows():
        indexNum = transactionFitCopy.index.get_loc(index)
        if first:
            first = False
            transactionFitCopy.iat[indexNum,1] = 100
            # print(i,transactionFitCopy.iat[i,1],"#1")
            continue
        if len(element.holding) == 0:
            transactionFitCopy.iat[indexNum,1] = transactionFitCopy.iat[indexNum-1,1]
            # print(i,transactionFitCopy.iat[i,1],"#2")
            continue
        else:
            prevData = transactionFitCopy.iat[indexNum-1, 0]
            if len(prevData):
                total = 0
                removeList = []
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            if element.holding[j][2] == -1:
                                removeList.append(element.holding[j])
                            break
                    total += close*prevData[i][2]
                transactionFitCopy.iat[indexNum,1] = total
                # print(i,transactionFitCopy.iat[indexNum,1],"#3")
                for delete in removeList:
                    if delete in element.holding:
                        element.holding.remove(delete)
            else:
                transactionFitCopy.iat[indexNum,1] = transactionFitCopy.iat[indexNum-1,1]
                # print(i,transactionFitCopy.iat[i,1],"#4")
        if len(element.holding):
            allocation = total/len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
    stockHolding  = 0
    for i in transactionFitCopy.iterrows():
        stockHolding += len(i[1]['holding'])
    if stockHolding/(transactionFitCopy.shape[0])<(len(tickers)/1000) or transactionFitCopy.iat[-1,TOTAL]<=100:
        print(stockHolding)
        print(transactionFitCopy.iloc[-1]['total'])
        return -1
    return transactionFitCopy.iloc[-1]['total']

In [6]:
evalOneMax([0.030044812102893892, 0.933142178892616, 0.3116970470712026, 1.0339563307107682, 1.2244313772350464, 1.2796475840086914, 0.9186099529632504, 0.8161387343584178, 1.242629912183577, 0.998882421088025, -0.08626025478563233, 0.937001716098529, 0.9928448924991358, 0.9037365566622244, 0.984357966000531])

[0.030044812102893892, 0.933142178892616, 0.3116970470712026, 1.0339563307107682, 1.2244313772350464, 1.2796475840086914, 0.9186099529632504, 0.8161387343584178, 1.242629912183577, 0.998882421088025, -0.08626025478563233, 0.937001716098529, 0.9928448924991358, 0.9037365566622244, 0.984357966000531]


KeyError: Timestamp('2022-04-01 00:00:00')